# API Model Introspection

Test introspection capabilities of frontier models (Claude, GPT-4) using prompt-based steering.

## Key Difference from MLX Introspection

**MLX Models**: True activation steering - We inject concept vectors directly into model activations.

**API Models**: Prompt-based steering - We simulate concept injection using system prompts.

## Why Test API Models?

1. **Frontier capability comparison** - Claude Opus 4 showed highest introspection in the paper
2. **Benchmark local models** - Compare your MLX models against SOTA
3. **Natural introspection** - Test if models can self-report without any steering
4. **Method comparison** - Does prompt-steering approximate activation steering?

## Setup

Make sure you have API keys set:
```bash
export ANTHROPIC_API_KEY="your-key"
export OPENAI_API_KEY="your-key"
```

In [ ]:
# 1. Add parent directory to path for code imports
import sys
sys.path.append('../code')

# 2. Import OS module to check for API keys
import os

# 3. Import harness functions for API-based introspection
from harness import (
    run_strategy,                        # Run introspection strategy
    APIIntrospectionTester,              # Test API models for introspection
    NATURAL_INTROSPECTION_PROMPTS        # Pre-defined introspection prompts
)

# 4. Check which API keys are available
has_anthropic = bool(os.getenv("ANTHROPIC_API_KEY"))
has_openai = bool(os.getenv("OPENAI_API_KEY"))

# 5. Print availability status
print(f"Anthropic API: {'✓' if has_anthropic else '✗ (set ANTHROPIC_API_KEY)'}")
print(f"OpenAI API: {'✓' if has_openai else '✗ (set OPENAI_API_KEY)'}")

## 1. Test Prompt-Based Steering

Simulate concept injection using system prompts.

In [ ]:
# 1. Test prompt-based steering with Claude (if API key available)
if has_anthropic:
    print("Testing Claude 3.5 Sonnet...\n")
    
    # 2. Run introspection with prompt-based concept steering
    result = run_strategy(
        "introspection",                    # Introspection strategy
        task_input="Describe a typical day at work",  # Neutral task
        concept="happiness",                 # Concept to simulate via prompting
        task_type="detection",               # Ask if concept is present
        provider="anthropic",                # Use Anthropic API
        model="claude-3-5-sonnet-20241022", # Specific Claude model
        api_strength="moderate",             # Strength of prompt-based steering
        steering_style="implicit",           # Subtle steering hints
        temperature=0.7,                     # Sampling temperature
        verbose=True                         # Show detailed output
    )
    
    # 3. Print results summary
    print("\n" + "="*60)
    print("RESULTS:")
    print("="*60)
    
    # 4. Did model detect the simulated injection?
    print(f"Detected injection: {result.metadata['introspection_correct']}")
    
    # 5. Confidence level of detection
    print(f"Confidence: {result.metadata['introspection_confidence']:.2f}")
else:
    print("Skipping - no Anthropic API key")

In [ ]:
# Test with GPT-4
if has_openai:
    print("Testing GPT-4o...\n")
    
    result = run_strategy(
        "introspection",
        task_input="Describe a typical day at work",
        concept="happiness",
        task_type="detection",
        provider="openai",
        model="gpt-4o",
        api_strength="moderate",
        steering_style="implicit",
        temperature=0.7,
        verbose=True
    )
    
    print("\n" + "="*60)
    print("RESULTS:")
    print("="*60)
    print(f"Detected injection: {result.metadata['introspection_correct']}")
    print(f"Confidence: {result.metadata['introspection_confidence']:.2f}")
else:
    print("Skipping - no OpenAI API key")

## 2. Test Natural Introspection

Can models report on their own reasoning without any steering?

In [ ]:
if has_anthropic:
    tester = APIIntrospectionTester(
        provider="anthropic",
        model="claude-3-5-sonnet-20241022"
    )
    
    # Test with a few natural introspection prompts
    test_prompts = NATURAL_INTROSPECTION_PROMPTS[:5]
    
    results = tester.test_natural_introspection(
        prompts=test_prompts,
        temperature=0.7,
        verbose=True
    )
    
    # Calculate total cost
    total_cost = sum(r.get('cost_usd', 0) for r in results)
    print(f"\nTotal cost: ${total_cost:.4f}")

## 3. Compare Steering Strengths

Test subtle, moderate, and strong prompt-based steering.

In [ ]:
if has_anthropic:
    strengths = ["subtle", "moderate", "strong"]
    
    for strength in strengths:
        print(f"\n{'='*60}")
        print(f"Testing strength: {strength}")
        print(f"{'='*60}")
        
        result = run_strategy(
            "introspection",
            task_input="Think about emotions",
            concept="anger",
            task_type="detection",
            provider="anthropic",
            model="claude-3-5-sonnet-20241022",
            api_strength=strength,
            steering_style="implicit",
            verbose=False
        )
        
        print(f"Detected: {result.metadata['introspection_correct']}")
        print(f"Confidence: {result.metadata['introspection_confidence']:.2f}")
        print(f"Response: {result.output[:150]}...")

## 4. Model Comparison

Compare introspection across different frontier models.

In [ ]:
import pandas as pd

models_to_test = []

if has_anthropic:
    models_to_test.extend([
        ("anthropic", "claude-3-5-sonnet-20241022"),
        ("anthropic", "claude-3-5-haiku-20241022"),
    ])

if has_openai:
    models_to_test.extend([
        ("openai", "gpt-4o"),
        ("openai", "gpt-4o-mini"),
    ])

results = []
concepts = ["happiness", "sadness", "anger"]

for provider, model in models_to_test:
    print(f"\nTesting {model}...")
    
    for concept in concepts:
        result = run_strategy(
            "introspection",
            task_input="Describe your current feelings",
            concept=concept,
            task_type="detection",
            provider=provider,
            model=model,
            api_strength="moderate",
            verbose=False
        )
        
        results.append({
            'provider': provider,
            'model': model,
            'concept': concept,
            'correct': result.metadata['introspection_correct'],
            'confidence': result.metadata['introspection_confidence']
        })

df = pd.DataFrame(results)
print("\n" + "="*60)
print("COMPARISON RESULTS")
print("="*60)
print(df.groupby('model').agg({'correct': 'mean', 'confidence': 'mean'}))

## 5. Implicit vs Explicit Steering

Compare implicit prompting (subtle hints) vs explicit prompting (direct statements).

In [ ]:
if has_anthropic:
    styles = ["implicit", "explicit"]
    
    for style in styles:
        print(f"\n{'='*60}")
        print(f"Testing style: {style}")
        print(f"{'='*60}")
        
        result = run_strategy(
            "introspection",
            task_input="Describe a memory",
            concept="happiness",
            task_type="detection",
            provider="anthropic",
            model="claude-3-5-sonnet-20241022",
            api_strength="moderate",
            steering_style=style,
            verbose=False
        )
        
        print(f"Detected: {result.metadata['introspection_correct']}")
        print(f"Steered response: {result.output[:200]}...")

## 6. Key Findings & Observations

### Expected Results (from paper):

- **Claude Opus 4**: Highest introspective awareness (~70-80% accuracy)
- **GPT-4**: Good introspection (~60-70% accuracy)
- **Smaller models**: Lower accuracy (~40-50%)

### Limitations of Prompt-Based Steering:

1. **Less precise** than activation steering
2. **Dependent on instruction following** - model must cooperate with system prompt
3. **Can't control layer depth** - only surface-level influence
4. **Cost** - API calls are expensive for large experiments

### When to Use API Testing:

✅ **Good for:**
- Benchmarking local models against SOTA
- Testing natural introspection (no steering needed)
- Quick validation of paper findings
- Comparing frontier model capabilities

❌ **Not good for:**
- Fine-grained layer analysis
- Causal mechanistic studies
- High-volume experiments (expensive)
- Research requiring activation-level control

### Research Value:

The combination of MLX (activation steering) + API (prompt steering) enables:
- **Method validation**: Does prompt steering approximate activation steering?
- **Capability ceiling**: How much better are frontier models?
- **Local model potential**: Can fine-tuning close the gap?


## Next Steps

1. **Compare MLX vs API**: Use `APIIntrospectionTester.compare_with_mlx()` to directly compare
2. **Larger study**: Test more models, concepts, and steering strengths
3. **Natural introspection**: Explore models' self-awareness without any steering
4. **Fine-tuning impact**: Does fine-tuning improve introspection?

See `INTROSPECTION.md` for full API documentation.